In [ ]:
#about 40% of the submission are based on new products
#therefore, need to create a dataset of the first month of sales per product
#then do EDA on it
    #what is the price variation of the variable
    #are the timing specifities - when bought in November has an impact
    #are the products that are to be realised all of the same category?
    #are the shops we need to predict all open?
    #how do shops tend to perform over the year?  Are there some that get a greater increase?